# Surface Brightness Temperature Functions

### Contains functions for generating 3d array of surface brightness temperatures from CCDC 

In [12]:
import os
import sys
import time
import xarray as xr
import multiprocessing as mp
import datetime as dt
from collections import defaultdict
from typing import Callable, Union, List, Tuple

import requests
import numpy as np
from datetime import date, timedelta
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import rasterio 

import rioxarray
import pyproj

In [13]:
#Affine transformation information
Affine = Tuple[int, int, int, int, int, int]

_cu_til_aff = (-2565585, 150000, 0, 3314805, 0, -150000)
_cu_chp_aff = (-2565585, 3000, 0, 3314805, 0, -3000)
_cu_pxl_aff = (-2565585, 30, 0, 3314805, 0, -30)

_ccdc_ceph = 'http://lsdslb.cr.usgs.gov:7484/ard-cu-c01-v01-aux-cu-v01-ccdc-1-3-copy'

### Helper functions for accessing CCDC

In [ ]:
def retry(retries: int) -> Callable:
    """
    Simple retry decorator, for retrying any function that may throw an exception
    such as when trying to retrieve network resources
    """
    def retry_dec(func: Callable) -> Callable:
        def wrapper(*args, **kwargs):
            count = 1
            while True:
                try:
                    return func(*args, **kwargs)
                except:
                    count += 1
                    if count > retries:
                        raise
                    time.sleep(5)
        return wrapper
    return retry_dec

@retry(5)
def get_json(resource: str, params: dict = None) -> Union[List, dict]:
    """
    Simple function for doing the base communication with the REST services
    """
    resp = requests.get(resource, params=params, headers={'Connection': 'close'})

    if not resp.ok:
        resp.raise_for_status()

    return resp.json()

def get_segments(cx: int, cy: int, resource: str = _ccdc_ceph) -> List[dict]:
    """
    Retrieve change segments
    """
    return get_json(f'{resource}/segment/{cx}-{cy}.json')

In [15]:
def contains(x: float, y: float, affine: Affine) -> Tuple[float, float]:
    """
    Determine the containing ulx/uly of the "cell" (based on the affine) for the given x/y
    """
    r, c = to_rowcol(x, y, affine)

    return to_geo(r, c, affine)

def to_geo(row: int, col: int, affine: Affine) -> Tuple[float, float]:
    """
    Transform a row/col into a geospatial coordinate given reference affine.
    Xgeo = GT(0) + Xpixel*GT(1) + Yline*GT(2)
    Ygeo = GT(3) + Xpixel*GT(4) + Yline*GT(5)
    """
    x = affine[0] + col * affine[1]
    y = affine[3] + row * affine[5]

    return x, y

def to_rowcol(x: int, y: int, affine: Affine) -> Tuple[int, int]:
    """
    Affine transformation from projected coordinates to row/col space
    """
    col = (x - affine[0]) / affine[1]
    row = (y - affine[3]) / affine[5]

    return int(row), int(col)

def contain_chip(x, y, chip_aff: Affine = _cu_chp_aff) -> Tuple[float, float]:
    """
    Determine the containing LCMAP chip ulx/uly
    """
    return contains(x, y, chip_aff)

def contain_pixel(x, y, pxl_aff: Affine = _cu_pxl_aff) -> Tuple[float, float]:
    """
    Determine the containing LCMAP chip ulx/uly
    """
    return contains(x, y, pxl_aff)

def chip_affine(cx: int, cy: int) -> Affine:
    """
    Build a 30m affine for a chip
    """
    return cx, 30, 0, cy, 0, -30

### Helper functions for generating CCDC values from harmonic model

In [16]:
def intersect_fuzzy(sorted_segs: List[dict], date: str) -> dict:
    """
    Select a model to build predictions from
    """
    if not sorted_segs:
        return {}

    # ord date before time series models -> cover back
    if date < sorted_segs[0]['sday']:
        return sorted_segs[0]

    # ord date after time series models -> cover forward
    if date > sorted_segs[-1]['eday']:
        return sorted_segs[-1]

    prev_br = None
    for seg in sorted_segs:
        # Date is contained within the model
        if seg['sday'] <= date < seg['bday']:
            return seg
        if prev_br and prev_br <= date <= seg['sday']:
            return seg

        prev_br = seg['bday']

    raise ValueError
    
def group_segments(segments: List[dict]) -> dict:
    """
    Group the segments by px/py, sorted by sday for easier iterating downstream
    """
    d = defaultdict(list)

    for s in segments:
        d[(s['px'], s['py'])].append(s)

    return {k: sorted(p, key=lambda x: x['sday']) for k, p in d.items()}


def to_ord(date: str) -> int:
    """
    Convert a date string to an ordinal value
    """
    return dt.datetime.strptime(date, '%Y-%m-%d').toordinal()


def predict(b_int: float, b_coefs: List[float], ordinal: int) -> float:
    """
    Predict a band value for the given date
    """
    w = 2 * np.pi / 365.2425
    sl, c1, c2, c3, c4, c5, c6 = b_coefs
    return (c1 * np.cos(w * ordinal) + c2 * np.sin(w * ordinal) +
            c3 * np.cos(2 * w * ordinal) + c4 * np.sin(2 * w * ordinal) +
            c5 * np.cos(3 * w * ordinal) + c6 * np.sin(3 * w * ordinal) +
            b_int + sl * ordinal)

def pxl_thrm(sorted_segs: List[dict], date: str) -> int:
    """
    Calculate predicted scaled brightness temp for the given date
    """
    ordinal = to_ord(date)
    seg = intersect_fuzzy(sorted_segs, date)

    thrm = predict(seg['thint'], seg['thcoef'], ordinal)
    #rescale thrm to kelvin
    return int((thrm+27315))

def chip_thrm(chip_data: dict, date: str, affine: Affine) -> np.ndarray:
    """
    Calculate synthetic brightness temp for a chip.
    """
    out = np.full(shape=(100, 100), dtype=int, fill_value=-9999)

    for coord, segments in chip_data.items():
        out[to_rowcol(*coord, affine)] = pxl_thrm(segments, date)

    return out

def get_xvals(xy_keys: list) -> list:
    """
    return sorted list of x values each pixel for given chip
    given list of all pixel choords
    """
    xlist = []
    for b, key in enumerate(xy_keys):
        xlist.append(key[0])

    xlist = list(set(xlist))  
    xlist.sort(reverse = False)
    return xlist

def get_yvals(xy_keys: list) -> list:
    """
    return sorted list of y values each pixel for given chip
    given list of all pixel choords
    """
    ylist = []
    for c, key in enumerate(xy_keys):
        ylist.append(key[1])

    ylist = list(set(ylist))  
    ylist.sort(reverse = False)
    return ylist

### Function to make synthetic chips and merge them into large 3d array

In [1]:
def super_grid(somex, somey, dates : list, numchips : int):
    
    """
    returns 3D array of synthetic thermal band values 
    along with lists of x and y values 
    
    Impute x, y vals from top left chip, list of dates,
    and number of chips to extend grid out by
    """

    #create holder x, y lists for lat lon 
    valsx = []
    valsy = []
    count = 1

    #yforloop
    chipsy = []

    for n in range(0, numchips):
        someyii = somey -(2970*n)
    
        #xfor loop
        chipsx = []
        for g in range(0, numchips):
            #Create holder list 
            thermlist  = []
            #get chip data
            some_coord = (somex+(2970*g), someyii)
            some_chip = contain_chip(*some_coord)
            chip_segments = get_segments(*some_chip)
            sorted_chip = group_segments(chip_segments)
            print(str(count) + " " + str(some_coord))
            #create synthetic ship for each date 
            for i, date in enumerate(dates):
                sdate = date.strftime('%Y-%m-%d')
                thermtemp = chip_thrm(sorted_chip, sdate, chip_affine(*some_chip)) #date must be string 'yyyy-mm-dd'
                thermlist.append(thermtemp)
                
    
            #put synethic chip into holder array 
            thermarray = np.dstack(thermlist)
            chipsx.append(thermarray)
            gridrow = np.hstack(chipsx)
    
            #add x and y values to valsx,y list 
            mykeys_list2 = list(sorted_chip.keys())
            valsx = valsx + get_xvals(mykeys_list2)
            valsy = valsy + get_yvals(mykeys_list2)
            #remove duplicate x and ys
            valsx = list(set(valsx))
            valsy = list(set(valsy))
            
            count = count + 1
    
        chipsy.append(gridrow)
        

    #create final array 
    farray = np.vstack(chipsy)
    farray2 = farray / 100


    return farray2, valsx, valsy